In [8]:
import os
from PyPDF2 import PdfReader, PdfWriter

def get_bookmark_page_number(reader, bookmark):
    """获取书签对应的页码"""
    try:
        return reader.get_destination_page_number(bookmark)
    except:
        try:
            # 尝试直接从页面对象获取页码
            if '/Page' in bookmark:
                page_ref = bookmark['/Page']
                for i, page in enumerate(reader.pages):
                    if page.indirect_reference == page_ref:
                        return i
        except:
            pass
    return None

def extract_bookmarks_recursive(reader):
    """提取所有书签"""
    bookmarks_list = []
    
    def process_bookmark(bookmark, level=0, parent=""):
        if isinstance(bookmark, dict):
            title = bookmark.get('/Title', 'Untitled')
            full_title = f"{parent} - {title}" if parent else title
            
            # 获取页码
            page_num = get_bookmark_page_number(reader, bookmark)
            if page_num is not None:
                print(f"找到书签: {full_title} (层级: {level}, 页码: {page_num})")
                bookmarks_list.append({
                    'title': full_title,
                    'page': page_num,
                    'level': level
                })
            else:
                print(f"警告：无法获取书签页码: {full_title}")
            
            # 处理子书签
            if '/First' in bookmark:
                process_bookmark(bookmark['/First'], level + 1, full_title)
            # 处理同级书签
            if '/Next' in bookmark:
                process_bookmark(bookmark['/Next'], level, parent)
        elif isinstance(bookmark, list):
            for b in bookmark:
                process_bookmark(b, level, parent)
    
    # 处理根书签
    outline = reader.outline
    if isinstance(outline, list):
        for item in outline:
            process_bookmark(item)
    else:
        process_bookmark(outline)
    
    return bookmarks_list

def split_pdf_by_bookmarks(input_pdf, output_dir, target_level=0):
    """根据指定层级的书签切分PDF文件"""
    print(f"正在处理PDF文件: {input_pdf}")
    print(f"输出目录: {output_dir}")
    print(f"目标书签层级: {target_level}")
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 读取PDF
    reader = PdfReader(input_pdf)
    print(f"PDF总页数: {len(reader.pages)}")
    
    # 获取所有书签
    all_bookmarks = extract_bookmarks_recursive(reader)
    
    if not all_bookmarks:
        print("未找到任何书签！")
        return
    
    # 筛选指定层级的书签
    target_bookmarks = [b for b in all_bookmarks if b['level'] == target_level]
    
    if not target_bookmarks:
        print(f"没有找到{target_level}级书签！")
        print("可用的书签层级:")
        levels = sorted(set(b['level'] for b in all_bookmarks))
        for level in levels:
            count = sum(1 for b in all_bookmarks if b['level'] == level)
            print(f"层级 {level}: {count} 个书签")
        return
    
    print(f"\n找到 {len(target_bookmarks)} 个目标层级书签:")
    for i, b in enumerate(target_bookmarks):
        print(f"{i+1}. {b['title']} (页码: {b['page']})")
    
    # 按书签切分PDF
    for i in range(len(target_bookmarks)):
        writer = PdfWriter()
        current = target_bookmarks[i]
        
        # 确定页码范围
        start_page = current['page']
        if i < len(target_bookmarks) - 1:
            end_page = target_bookmarks[i + 1]['page']
        else:
            end_page = len(reader.pages)
        
        print(f"\n正在处理 '{current['title']}'")
        print(f"页码范围: {start_page} 到 {end_page-1}")
        
        # 复制页面
        try:
            for page_num in range(start_page, end_page):
                if page_num < len(reader.pages):
                    writer.add_page(reader.pages[page_num])
                    print(f"已添加页面 {page_num}", end='\r')
            
            # 生成文件名
            safe_title = "".join(c for c in current['title'] if c.isalnum() or c in (' ', '-', '_'))
            output_file = os.path.join(output_dir, f"{i+1:03d}-{safe_title}.pdf")
            
            # 保存文件
            with open(output_file, 'wb') as output:
                writer.write(output)
            print(f"\n已创建: {output_file}")
        except Exception as e:
            print(f"处理文件时出错: {e}")

if __name__ == "__main__":
    input_pdf = "文化研究关键词 - 汪民安.pdf"
    output_dir = "split_pdfs"
    
    # 使用层级 0 来切分（对应第一级书签）
    split_pdf_by_bookmarks(input_pdf, output_dir, target_level=0)

正在处理PDF文件: 文化研究关键词 - 汪民安.pdf
输出目录: split_pdfs
目标书签层级: 0
PDF总页数: 946
找到书签: 版权信息 (层级: 0, 页码: 1)
找到书签: 修订版前言 (层级: 0, 页码: 11)
找到书签: 前言：词语的深渊 (层级: 0, 页码: 13)
找到书签: B (层级: 0, 页码: 18)
找到书签: 保守主义（Conservatism） (层级: 0, 页码: 18)
找到书签: 暴力（Violence） (层级: 0, 页码: 24)
找到书签: 辩证法（Dialectic） (层级: 0, 页码: 28)
找到书签: 辩证意象（Dialectic Image） (层级: 0, 页码: 30)
找到书签: C (层级: 0, 页码: 34)
找到书签: 擦抹（Under Erasure） (层级: 0, 页码: 34)
找到书签: 残酷戏剧（le théatre de la cruauté） (层级: 0, 页码: 38)
找到书签: 策略性本质主义（Strategic Essentialism） (层级: 0, 页码: 43)
找到书签: 差异（Difference） (层级: 0, 页码: 47)
找到书签: 场域（Field） (层级: 0, 页码: 53)
找到书签: 超人（übermensch） (层级: 0, 页码: 57)
找到书签: 超真实（Hyperreal） (层级: 0, 页码: 60)
找到书签: 赤裸生命（Bare Life） (层级: 0, 页码: 64)
找到书签: 词语误用（Catachresis） (层级: 0, 页码: 68)
找到书签: 存在（Being） (层级: 0, 页码: 71)
找到书签: 存在主义（Existentialism） (层级: 0, 页码: 76)
找到书签: D (层级: 0, 页码: 80)
找到书签: 单面人（OneDimensionalMan） (层级: 0, 页码: 80)
找到书签: 单子（Monad） (层级: 0, 页码: 84)
找到书签: 地方性知识（Local Knowledge） (层级: 0, 页码: 87)
找到书签: 帝国（Empire） (层级: 0, 页码: 90)
找到书签: 第三空间（Third Spa

# epub

In [14]:
from ebooklib import epub
import html2text

def extract_epub_chapters(epub_path):
    """
    从epub文件中提取章节内容
    参数:
        epub_path: epub文件的路径
    返回:
        chapters: 包含章节标题和内容的字典列表
    """
    # 读取epub文件
    book = epub.read_epub(epub_path)
    
    # 创建HTML到文本的转换器
    h = html2text.HTML2Text()
    h.ignore_links = True
    h.ignore_images = True
    
    chapters = []
    
    # 遍历所有文档
    for item in book.get_items():
        # 检查是否为文档类型（使用正确的常量）
        if isinstance(item, epub.EpubHtml):
            # 获取文档内容
            content = item.get_content().decode('utf-8')
            # 将HTML转换为纯文本
            text = h.handle(content)
            
            # 获取章节标题
            title = item.title if hasattr(item, 'title') else None
            
            chapters.append({
                'title': title or f"Chapter {len(chapters) + 1}",
                'content': text
            })
    
    return chapters

def save_chapters_to_files(chapters, output_dir):
    """
    将提取的章节保存为单独的文本文件
    参数:
        chapters: 章节列表
        output_dir: 输出目录路径
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    for i, chapter in enumerate(chapters):
        # 清理文件名，移除不合法字符
        if chapter['title']:
            filename = "".join(x for x in chapter['title'] if x.isalnum() or x in (' ', '-', '_'))
            filename = f"{filename}.txt"
        else:
            filename = f"chapter_{i+1}.txt"
        
        filepath = os.path.join(output_dir, filename)
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(chapter['content'])
        print(f"已保存: {filepath}")

# 使用示例
if __name__ == "__main__":
    epub_path = "文化研究关键词 (汪民安) (Z-Library).epub"  # 替换为你的epub文件路径
    output_dir = "chapters"       # 输出目录
    
    chapters = extract_epub_chapters(epub_path)
    save_chapters_to_files(chapters, output_dir)

已保存: chapters\Chapter 1.txt
已保存: chapters\Chapter 2.txt
已保存: chapters\Chapter 3.txt
已保存: chapters\Chapter 4.txt
已保存: chapters\Chapter 5.txt
已保存: chapters\Chapter 6.txt
已保存: chapters\Chapter 7.txt
已保存: chapters\Chapter 8.txt
已保存: chapters\Chapter 9.txt
已保存: chapters\Chapter 10.txt
已保存: chapters\Chapter 11.txt
已保存: chapters\Chapter 12.txt
已保存: chapters\Chapter 13.txt
已保存: chapters\Chapter 14.txt
已保存: chapters\Chapter 15.txt
已保存: chapters\Chapter 16.txt
已保存: chapters\Chapter 17.txt
已保存: chapters\Chapter 18.txt
已保存: chapters\Chapter 19.txt
已保存: chapters\Chapter 20.txt
已保存: chapters\Chapter 21.txt
已保存: chapters\Chapter 22.txt
已保存: chapters\Chapter 23.txt
已保存: chapters\Chapter 24.txt
已保存: chapters\Chapter 25.txt
已保存: chapters\Chapter 26.txt
已保存: chapters\Chapter 27.txt
已保存: chapters\Chapter 28.txt
已保存: chapters\Chapter 29.txt
已保存: chapters\Chapter 30.txt
已保存: chapters\Chapter 31.txt
已保存: chapters\Chapter 32.txt
已保存: chapters\Chapter 33.txt
已保存: chapters\Chapter 34.txt
已保存: chapters\Chapter 3

In [17]:
import json
import re
import os

def process_chapter_content(text):
    """
    处理单个章节内容，提取二级标题和段落
    返回: 字典，键为二级标题，值为段落列表
    """
    # 寻找二级标题（以##开头）
    title_match = re.search(r'^##\s+(.+)$', text, re.MULTILINE)
    if not title_match:
        return {}
    
    title = title_match.group(1).strip()
    
    # 获取标题后的内容
    content = text[title_match.end():].strip()
    
    # 用连续换行符分割段落
    paragraphs = [p.strip() for p in re.split(r'\n\s*\n', content)]
    
    # 过滤掉长度小于10的段落
    valid_paragraphs = [p.replace('\n','') for p in paragraphs if len(p) >= 10]
    
    return {title: valid_paragraphs}

def process_all_chapters(directory):
    """
    处理目录中的所有txt文件
    参数:
        directory: chapters目录的路径
    返回:
        包含所有章节内容的字典
    """
    result = {}
    
    # 遍历目录中的所有txt文件
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            try:
                # 读取文件内容
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                # 处理章节内容
                chapter_dict = process_chapter_content(content)
                # 更新结果字典
                result.update(chapter_dict)
                
            except Exception as e:
                print(f"处理文件 {filename} 时出错: {str(e)}")
    
    return result

def save_to_json(data, output_file):
    """
    将数据保存为JSON文件
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# 使用示例
if __name__ == "__main__":
    chapters_dir = "chapters"  # 指定chapters目录的路径
    output_file = "chapters_content.json"  # 输出JSON文件的路径
    
    # 处理所有章节
    result = process_all_chapters(chapters_dir)
    print(len(result))
    # 保存为JSON文件
    save_to_json(result, output_file)
    
    print(f"处理完成，结果已保存到 {output_file}")

226
处理完成，结果已保存到 chapters_content.json


# 总结

In [20]:
import json
import logging
import time
import os
from typing import List, Dict, Optional, Union
from zhipuai import ZhipuAI


def setup_logging():
    """
    配置双重日志系统：控制台输出和错误文件记录
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 创建logs目录
    os.makedirs('logs', exist_ok=True)

    # 控制台处理器
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(console_formatter)

    # 错误日志文件处理器
    error_handler = logging.FileHandler('logs/error.log', encoding='utf-8')
    error_handler.setLevel(logging.ERROR)
    error_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s\n详细信息：%(exc_info)s')
    error_handler.setFormatter(error_formatter)

    logger.addHandler(console_handler)
    logger.addHandler(error_handler)


class TextSummarizer:
    """
    文本总结器，负责生成总结并保存相关文件
    """

    def __init__(self, api_key: str, model: str = "glm-4"):
        """
        初始化总结器

        Args:
            api_key: 智谱AI的API密钥
            model: 使用的模型名称
        """
        self.client = ZhipuAI(api_key=api_key)
        self.model = model
        self.summaries_dir = "summaries"
        os.makedirs(self.summaries_dir, exist_ok=True)
        logging.info(f"初始化TextSummarizer完成，使用模型: {model}")

    def create_summary(self, text: str, key: str, max_retries: int = 3) -> Optional[str]:
        """
        使用智谱AI为文本生成总结

        Args:
            text: 需要总结的文本内容
            key: 关键词/主题
            max_retries: 最大重试次数
        """
        retry_count = 0
        while retry_count < max_retries:
            try:
                prompt = f"请针对以下与'{key}'相关的回答，猜想该回答的对应问题是什么？"
                text = text.replace(' ','').replace('\n','')
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {
                            "role": "system",
                            "content": "你是一位平易近人的文化研究教授，请猜想下面的回答对应最重要问题是什么？用简短的语言，只返回问题内容。"
                        },
                        {
                            "role": "user",
                            "content": f"{prompt}\n\n{text}"
                        }
                    ],
                    temperature=0.7,
                    top_p=0.7,
                    max_tokens=1024,
                    stream=False
                )

                if hasattr(response, 'choices') and len(response.choices) > 0:
                    summary = response.choices[0].message.content
                    logging.info(f"成功生成总结: {summary}")
                    return summary.strip()

            except Exception as e:
                retry_count += 1
                logging.error(f"生成总结时发生错误: {str(e)}", exc_info=True)
                if retry_count < max_retries:
                    time.sleep(2 * retry_count)
                continue

        return None

    def save_summary_and_text(self, summary: str, text: str, key: str, index: int) -> str:
        """
        将总结和原文保存到txt文件

        Args:
            summary: 生成的总结
            text: 原始文本
            key: 关键词/主题
            index: 文本块索引
        """
        safe_key = "".join(c for c in key if c.isalnum() or c in (' ', '-', '_')).strip()
        filename = f"{safe_key}_chunk_{index}.txt"
        filepath = os.path.join(self.summaries_dir, filename)

        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(f"总结：\n{summary}\n\n原文：\n{text}")
            logging.info(f"已保存文件: {filepath}")
            return filepath
        except Exception as e:
            logging.error(f"保存文件时发生错误: {str(e)}", exc_info=True)
            return ""


class AlpacaFormatter:
    """
    负责将处理结果转换为Alpaca格式
    """

    @staticmethod
    def convert_to_alpaca(data: Dict[str, Dict]) -> List[Dict]:
        """
        将数据转换为Alpaca训练格式

        Args:
            data: 原始数据字典
        """
        alpaca_data = []

        for key, content in data.items():
            if "processed_chunks" in content:
                for item in content["processed_chunks"]:
                    alpaca_entry = {
                        "instruction": item["summary"],
                        "input": "",
                        "output": item["original"],
                        "metadata": {
                            "key": key,
                            "file_path": item.get("file_path", ""),
                            "length": item.get("length", 0)
                        }
                    }
                    alpaca_data.append(alpaca_entry)

        return alpaca_data

def process_content(input_file: str, output_file: str, summarizer: TextSummarizer):
    """
    主处理函数，读取JSON文件并生成总结
    
    Args:
        input_file: 输入JSON文件路径
        output_file: 输出JSON文件路径
        summarizer: TextSummarizer实例
    """
    try:
        # 读取输入文件
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        processed_data = {}
        
        # 处理每个关键词的内容
        for key, chunks in data.items():
            logging.info(f"正在处理关键词: {key}")
            results = []
            
            # 直接遍历列表中的文本块
            for i, chunk in enumerate(chunks, 1):
                logging.info(f"正在处理 {key} 的第 {i}/{len(chunks)} 个文本块")
                
                # 生成总结
                summary = summarizer.create_summary(chunk, key)
                if summary:
                    # 保存到文件
                    file_path = summarizer.save_summary_and_text(summary, chunk, key, i)
                    
                    results.append({
                        "summary": summary,
                        "length": len(chunk),
                        "file_path": file_path
                    })
                
                time.sleep(1)  # 避免API限制
            
            # 保存处理结果
            processed_data[key] = {
                "original": chunks,  # 保存原始列表
                "processed_chunks": results,
                "metadata": {
                    "chunk_count": len(results),
                    "total_length": sum(len(chunk) for chunk in chunks)
                }
            }
            
            # 保存中间结果
            with open(f"{output_file}.temp", 'w', encoding='utf-8') as f:
                json.dump(processed_data, f, ensure_ascii=False, indent=2)
        
        # 转换为Alpaca格式
        alpaca_data = AlpacaFormatter.convert_to_alpaca(processed_data)
        
        # 保存最终结果
        alpaca_output = output_file.replace('.json', '_alpaca.json')
        with open(alpaca_output, 'w', encoding='utf-8') as f:
            json.dump(alpaca_data, f, ensure_ascii=False, indent=2)
        
        logging.info(f"处理完成，结果已保存至: {alpaca_output}")
    
    except Exception as e:
        logging.error(f"处理文件时发生错误: {str(e)}", exc_info=True)
        raise
    
def main():
    """
    主函数，配置并启动处理流程
    """
    # 设置日志系统
    setup_logging()

    # 配置参数
    API_KEY = "9d1abe1ba97fe0d44c1d62ab171ab6b1.48MgiTMFTb7GKo9U"
    MODEL = "glm-4-flash"
    INPUT_FILE = "chapters_content.json"
    OUTPUT_FILE = "output.json"

    try:
        summarizer = TextSummarizer(API_KEY, MODEL)
        process_content(INPUT_FILE, OUTPUT_FILE, summarizer)

    except Exception as e:
        logging.error(f"程序执行出错: {str(e)}", exc_info=True)
        raise


if __name__ == "__main__":
    main()

2024-12-16 15:18:54,940 - INFO - 初始化TextSummarizer完成，使用模型: glm-4-flash
2024-12-16 15:18:54,940 - INFO - 初始化TextSummarizer完成，使用模型: glm-4-flash
2024-12-16 15:18:54,940 - INFO - 初始化TextSummarizer完成，使用模型: glm-4-flash
2024-12-16 15:18:54,977 - INFO - 正在处理关键词: 擦抹（Under Erasure）
2024-12-16 15:18:54,977 - INFO - 正在处理关键词: 擦抹（Under Erasure）
2024-12-16 15:18:54,977 - INFO - 正在处理关键词: 擦抹（Under Erasure）
2024-12-16 15:18:54,980 - INFO - 正在处理 擦抹（Under Erasure） 的第 1/4 个文本块
2024-12-16 15:18:54,980 - INFO - 正在处理 擦抹（Under Erasure） 的第 1/4 个文本块
2024-12-16 15:18:54,980 - INFO - 正在处理 擦抹（Under Erasure） 的第 1/4 个文本块
2024-12-16 15:18:55,842 - INFO - HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
2024-12-16 15:18:55,842 - INFO - HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
2024-12-16 15:18:55,842 - INFO - HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
2024-12-16 15:18:55,849 - INFO 

KeyboardInterrupt: 

In [22]:
import json
import logging
import time
import os
from typing import List, Dict, Optional, Union
from zhipuai import ZhipuAI


def setup_logging():
    """
    配置双重日志系统：控制台输出和错误文件记录
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 创建logs目录
    os.makedirs('logs', exist_ok=True)

    # 控制台处理器
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(console_formatter)

    # 错误日志文件处理器
    error_handler = logging.FileHandler('logs/error.log', encoding='utf-8')
    error_handler.setLevel(logging.ERROR)
    error_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s\n详细信息：%(exc_info)s')
    error_handler.setFormatter(error_formatter)

    logger.addHandler(console_handler)
    logger.addHandler(error_handler)


class TextSummarizer:
    """
    文本总结器，负责生成总结并保存相关文件
    """

    def __init__(self, api_key: str, model: str = "glm-4"):
        """
        初始化总结器

        Args:
            api_key: 智谱AI的API密钥
            model: 使用的模型名称
        """
        self.client = ZhipuAI(api_key=api_key)
        self.model = model
        self.summaries_dir = "summaries"
        os.makedirs(self.summaries_dir, exist_ok=True)
        logging.info(f"初始化TextSummarizer完成，使用模型: {model}")

    def create_summary(self, text: str, key: str, max_retries: int = 3) -> Optional[str]:
        """
        使用智谱AI为文本生成总结

        Args:
            text: 需要总结的文本内容
            key: 关键词/主题
            max_retries: 最大重试次数
        """
        for retry_count in range(max_retries):
            try:
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[{
                        "role": "system",
                        "content": "你是一位平易近人的文化研究教授，请猜想下面的回答对应最重要问题是什么？用简短的语言，只返回问题内容。"
                    }, {
                        "role": "user",
                        "content": f"请针对以下与'{key}'相关的回答，猜想该回答的对应问题是什么？\n\n{text}"
                    }],
                    temperature=0.7,
                    top_p=0.7,
                    max_tokens=1024,
                    stream=False
                )
                
                # 成功获取响应就直接返回
                if hasattr(response, 'choices') and response.choices:
                    summary = response.choices[0].message.content.strip()
                    logging.info(f"成功生成总结: {summary}")
                    return summary
                
                # 如果没有有效响应，记录错误但继续重试
                logging.error(f"API返回无效响应: {response}")
                
            except Exception as e:
                # 只在最后一次重试失败时记录错误
                if retry_count == max_retries - 1:
                    logging.error(f"生成总结失败，已重试{retry_count + 1}次: {str(e)}", exc_info=True)
                elif retry_count < max_retries - 1:
                    logging.warning(f"第{retry_count + 1}次尝试失败，准备重试: {str(e)}")
                    time.sleep(2 * (retry_count + 1))  # 递增等待时间
        
        return None

    def save_summary_and_text(self, summary: str, text: str, key: str, index: int) -> str:
        """
        将总结和原文保存到txt文件

        Args:
            summary: 生成的总结
            text: 原始文本
            key: 关键词/主题
            index: 文本块索引
        """
        safe_key = "".join(c for c in key if c.isalnum() or c in (' ', '-', '_')).strip()
        filename = f"{safe_key}_chunk_{index}.txt"
        filepath = os.path.join(self.summaries_dir, filename)

        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(f"总结：\n{summary}\n\n原文：\n{text}")
            logging.info(f"已保存文件: {filepath}")
            return filepath
        except Exception as e:
            logging.error(f"保存文件时发生错误: {str(e)}", exc_info=True)
            return ""


class AlpacaFormatter:
    """
    负责将处理结果转换为Alpaca格式
    """

    @staticmethod
    def convert_to_alpaca(data: Dict[str, Dict]) -> List[Dict]:
        """
        将数据转换为Alpaca训练格式

        Args:
            data: 原始数据字典
        """
        alpaca_data = []

        for key, content in data.items():
            if "processed_chunks" in content:
                for item in content["processed_chunks"]:
                    alpaca_entry = {
                        "instruction": item["summary"],
                        "input": "",
                        "output": item["original"],
                        "metadata": {
                            "key": key,
                            "file_path": item.get("file_path", ""),
                            "length": item.get("length", 0)
                        }
                    }
                    alpaca_data.append(alpaca_entry)

        return alpaca_data


def process_content(input_file: str, output_file: str, summarizer: TextSummarizer):
    """
    主处理函数，读取JSON文件并生成总结
    
    Args:
        input_file: 输入JSON文件路径
        output_file: 输出JSON文件路径
        summarizer: TextSummarizer实例
    """
    try:
        # 读取输入文件
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        processed_data = {}
        
        # 处理每个关键词的内容
        for key, chunks in data.items():
            logging.info(f"正在处理关键词: {key}")
            results = []
            
            # 直接遍历列表中的文本块
            for i, chunk in enumerate(chunks, 1):
                logging.info(f"正在处理 {key} 的第 {i}/{len(chunks)} 个文本块")
                
                # 生成总结
                summary = summarizer.create_summary(chunk, key)
                if summary:
                    # 保存到文件
                    file_path = summarizer.save_summary_and_text(summary, chunk, key, i)
                    
                    results.append({
                        "original": chunk,
                        "summary": summary,
                        "length": len(chunk),
                        "file_path": file_path
                    })
                
                time.sleep(1)  # 避免API限制
            
            # 保存处理结果
            processed_data[key] = {
                "original": chunks,  # 保存原始列表
                "processed_chunks": results,
                "metadata": {
                    "chunk_count": len(results),
                    "total_length": sum(len(chunk) for chunk in chunks)
                }
            }
            
            # 保存中间结果
            with open(f"{output_file}.temp", 'w', encoding='utf-8') as f:
                json.dump(processed_data, f, ensure_ascii=False, indent=2)
        
        # 转换为Alpaca格式
        alpaca_data = AlpacaFormatter.convert_to_alpaca(processed_data)
        
        # 保存最终结果
        alpaca_output = output_file.replace('.json', '_alpaca.json')
        with open(alpaca_output, 'w', encoding='utf-8') as f:
            json.dump(alpaca_data, f, ensure_ascii=False, indent=2)
        
        logging.info(f"处理完成，结果已保存至: {alpaca_output}")
    
    except Exception as e:
        logging.error(f"处理文件时发生错误: {str(e)}", exc_info=True)
        raise


def main():
    """
    主函数，配置并启动处理流程
    """
    # 设置日志系统
    setup_logging()

    # 配置参数
    API_KEY = "9d1abe1ba97fe0d44c1d62ab171ab6b1.48MgiTMFTb7GKo9U"  # 替换为你的API密钥
    MODEL = "glm-4-flash"
    INPUT_FILE = "chapters_content.json"
    OUTPUT_FILE = "output.json"

    try:
        summarizer = TextSummarizer(API_KEY, MODEL)
        process_content(INPUT_FILE, OUTPUT_FILE, summarizer)

    except Exception as e:
        logging.error(f"程序执行出错: {str(e)}", exc_info=True)
        raise


if __name__ == "__main__":
    main()

2024-12-16 15:25:26,134 - INFO - 初始化TextSummarizer完成，使用模型: glm-4-flash
2024-12-16 15:25:26,134 - INFO - 初始化TextSummarizer完成，使用模型: glm-4-flash
2024-12-16 15:25:26,134 - INFO - 初始化TextSummarizer完成，使用模型: glm-4-flash
2024-12-16 15:25:26,134 - INFO - 初始化TextSummarizer完成，使用模型: glm-4-flash
2024-12-16 15:25:26,167 - INFO - 正在处理关键词: 擦抹（Under Erasure）
2024-12-16 15:25:26,167 - INFO - 正在处理关键词: 擦抹（Under Erasure）
2024-12-16 15:25:26,167 - INFO - 正在处理关键词: 擦抹（Under Erasure）
2024-12-16 15:25:26,167 - INFO - 正在处理关键词: 擦抹（Under Erasure）
2024-12-16 15:25:26,173 - INFO - 正在处理 擦抹（Under Erasure） 的第 1/4 个文本块
2024-12-16 15:25:26,173 - INFO - 正在处理 擦抹（Under Erasure） 的第 1/4 个文本块
2024-12-16 15:25:26,173 - INFO - 正在处理 擦抹（Under Erasure） 的第 1/4 个文本块
2024-12-16 15:25:26,173 - INFO - 正在处理 擦抹（Under Erasure） 的第 1/4 个文本块
2024-12-16 15:25:27,374 - INFO - HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
2024-12-16 15:25:27,374 - INFO - HTTP Request: POST https://open.bigmodel.cn/api